# Cortical Actin Quantification

## Imports

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

## Extract Boundaries from Images

Videos avi files. Drift correction performed already using StackReg package on ImageJ. 

In [ ]:
# Input: filename for a video
# Output: Arrays of binarized images for (1) cortical actin and (2) entire cell

def extractData(fileName):
    
    # Array of binarized image of cortical actin for each frame
    CortActinSeries = []
    
    # Array of binarized image of whole cell for each frame
    AllCellSeries = []
    
    # Array of contours for cortical actin for each frame
    ContoursSeries = []

    # Extract video information
    cap = cv2.VideoCapture(fileName)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Make object to save the video
#     saveName = fileName[:-4] + 'Modified.avi'  # change the file name if needed
#     imgSize = (height * 2, width)
#     frame_per_second = fps
#     writer = cv2.VideoWriter(saveName, cv2.VideoWriter_fourcc(*'DIVX'), frame_per_second, imgSize)

    # Loop through all frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:

            # Convert color to gray, remove timer/scale bar, erode, binarize, save
            grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            for i,j in product(range(50), range(100)):
                grayImg[i, j] = 0
            for i,j in product(range(height - 50, height), range(width - 200, width)):
                grayImg[i, j] = 0
            kernel = np.ones((3, 3), np.uint8)
            blurImg = cv2.GaussianBlur(grayImg,(3,3),0)
            erodeImg = cv2.erode(blurImg, kernel, iterations=1)
            
            # Cortical Actin - 30 set as threshold for all images
            ret, binImg = cv2.threshold(erodeImg,30,255,cv2.THRESH_BINARY)
            CortActinSeries.append(binImg)
            
            # Entire Cell
            ret, binImgAll = cv2.threshold(erodeImg,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            AllCellSeries.append(binImgAll)

            # Contours for Cortical Actin
            contours, hierarchy = cv2.findContours(binImg, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            ContoursSeries.append(contours)
            
            # Display original and binarized images with contours
            binImgColor = cv2.cvtColor(binImg,cv2.COLOR_GRAY2RGB)
            cv2.drawContours(binImgColor, contours, -1, (0,255,0), 0)
            combined = np.concatenate((frame, binImgColor), axis = 1)
#             writer.write(combined)
            cv2.imshow('frame',combined)
            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
#     writer.release()
    cv2.waitKey(1)
    
    return CortActinSeries, AllCellSeries, length, width, height

## Quantify Difference Across Frames

In [ ]:
# Input: Arrays of binarized images for (1) cortical actin and (2) entire cell + length/width/height of images
# Ouput: Representative metrics: avDiff is average number of pixels changing between frames, avActin is average 
# of pixels classified as actin, and avCell is average number of pixels classified as cell

def quantifyChange(CortActinSeries, AllCellSeries, length, width, height):
    avDiff = []
    avActin = []
    avCell = []
    for k in range(length - 1):
        diffs = 0 
        totalCortActin = 0
        totalCell = 0
        # Cosnider images in pairs
        for i,j in product(range(height), range(width)):
            currentVal = CortActinSeries[k][i, j]
            nextVal = CortActinSeries[k + 1][i, j]
            if currentVal != nextVal:
                diffs += 1
            if currentVal == 255:
                totalCortActin += 1
            if AllCellSeries[k][i, j] == 255:
                totalCell += 1
        avDiff.append(diffs)
        avActin.append(totalCortActin)
        avCell.append(totalCell)
    return np.mean(avDiff), np.mean(avActin), np.mean(avCell)

## Visualize Actin Across all Frames

In [ ]:
# Inputs: Array of binarized images for cortical actin and image name
# Output: Saves figure with cumulative actin at each location over time series at designated location

def makePlots(CortActinSeries, image):
    total = np.zeros(CortActinSeries[0].shape)
    for n in CortActinSeries:
        total += n
    plt.clf()
    plt.figure(figsize=(8, 8))
    plt.contourf(total, 100, cmap=plt.cm.nipy_spectral)
    plt.title(image[:-4])
    plt.axis('off')
#     plt.colorbar()
    plt.savefig("Cortical Actin/Figures/Superimposed " + image[:-4] + ".png", dpi = 300)

## Main Function

In [ ]:
def mainFunction(path, images):
    avDiffAll = []
    avActinAll = []
    avCellAll = []
    for image in images:
        if '.avi' in image and 'Modified' not in image:
            print(image)
            CortActinSeries, AllCellSeries, length, width, height = extractData(path + '/' + image)
            avDiff, avActin, avCell = quantifyChange(CortActinSeries, AllCellSeries, length, width, height)
            makePlots(CortActinSeries, image)
            avDiffAll.append(avDiff)
            avActinAll.append(avActin)
            avCellAll.append(avCell)
    return avDiffAll, avActinAll, avCellAll

## Dancing Project Data

### Day 0 Lifeact

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
avDiffAll_SG0, avActinAll_SG0, avCellAll_SG0 = mainFunction(path, images)

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
avDiffAll_CG0, avActinAll_CG0, avCellAll_CG0 = mainFunction(path, images)

### Day 4 Lifeact

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
avDiffAll_SG4, avActinAll_SG4, avCellAll_SG4 = mainFunction(path, images)

### Dataframe for All

In [ ]:
# SG D0 
data = {'SG0 Differences': avDiffAll_SG0, 'Total Actin': avActinAll_SG0, 'Total Cell': avCellAll_SG0}
df_SG_D0 = pd.DataFrame(data)
# CG D0
data = {'CG0 Differences': avDiffAll_CG0, 'Total Actin': avActinAll_CG0, 'Total Cell': avCellAll_CG0}
df_CG_D0 = pd.DataFrame(data)
# SG D4
data = {'SG4 Differences': avDiffAll_SG4, 'Total Actin': avActinAll_SG4, 'Total Cell': avCellAll_SG4}
df_SG_D4 = pd.DataFrame(data)
# Combine All data
allData = pd.concat([df_SG_D0, df_CG_D0, df_SG_D4], axis=1)
allData.to_csv("Cortical Actin/Final Data/allData.csv")

In [ ]:
allData